In [60]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException, TimeoutException
from selenium.webdriver.common.keys import Keys
import time


def get_bus_distance(homeAddress, officeAddress):
    # Initialize the WebDriver
    driver = webdriver.Chrome()  # or webdriver.Firefox(), etc.

    try:
        # Open Google Maps
        driver.get("https://www.google.ro/maps/preview/")
        
        # Function to click an element with fallback to JavaScript
        def click_element(locator):
            try:
                element = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable(locator)
                )
                element.click()
                
            except (ElementClickInterceptedException, ElementNotInteractableException):
                driver.execute_script("arguments[0].click();", element)
                print("Button clicked using JavaScript!")
            except NoSuchElementException:
                print("Element not found.")

        # Accept the cookies button
        try:
            accept_button_locator = (By.XPATH, "//button[.//span[text()='Accept all']]")
            click_element(accept_button_locator)
        except NoSuchElementException:
            print("Accept button not found.")

        # Input the address in the search box
        try:
            search_input = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.ID, "searchboxinput"))
            )
            search_input.clear()
            search_input.send_keys(officeAddress)
            

            # Click the search button
            search_button_locator = (By.ID, "hArJGc")
            click_element(search_button_locator)
            

            # Input a different address
            input_field = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "tactile-searchbox-input"))
            )
            input_field.clear()
            input_field.send_keys(homeAddress)
            
            input_field.send_keys(Keys.RETURN)
            WebDriverWait(driver,10)
            

            # Wait for the travel mode div to be visible and extract text
            travel_mode_div = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.XPATH, '//div[@data-travel_mode="3"]'))
            )
        
            try:
                button = travel_mode_div.find_element(By.TAG_NAME, 'button')  # Locate the button
                button.click()  # Click the button
                # print("Button inside the travel mode div clicked successfully!")
            except NoSuchElementException:
                print("Button not found inside the travel mode div.")
            except Exception as e:
                print(f"Error while clicking the button: {e}")
            
            time.sleep(3)
            travel_mode_text = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.XPATH, '//div[@data-travel_mode="3"]'))
            )
            travel_mode_text = travel_mode_text.text.split('\n')[1]
            return travel_mode_text
        except TimeoutException as te:
            print(f"Timeout Exception: {te}")
    except Exception as e:
        print(f"Error: {e}")
    driver.quit()


# home_address = "Bulevardul Tineretului, Tineretului, Sectorul 4, Bucuresti"
# # rentData=pd.read_csv("chirii.csv")
# office_address = "Calea Floreasca 246c, București 077190"
# # for index, record in rentData.iterrows():    
# #     rentData.at[index,"distance"]=get_bus_distance(record["adresa"], office_address)
# #     print(f"At index: {index} the distance between {office_address} and {record["adresa"]} is {rentData.at[index,"distance"]}")
# # rentData.to_csv("cleanRentData.csv")
# print(get_bus_distance(home_address,office_address))



In [61]:
def get_rent_links(nrOfPage,nrOfElemns,city):
    driver = webdriver.Chrome()
    # driver.get("https://www.storia.ro/")
    city=city.lower().strip()
    print(city)
    city=city.replace(" ","-")
    print(city)
    driver.get(f'https://www.storia.ro/ro/rezultate/inchiriere/apartament/alba-iulia?ownerTypeSingleSelect=ALL&viewType=listing&page={nrOfPage}')
    if driver.current_url.split("?")[1]=="fromInvalidLocation=true":
        print("WRONG CITY FORMAT")
        return -1
    try:
        accept_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))
        )
        click_with_fallback(accept_button, "Accept cookies button",driver)
    except TimeoutException:
        print("No 'Accept' button found or already dismissed.")

    # input_field = WebDriverWait(driver, 10).until(
    #     EC.presence_of_element_located((By.CSS_SELECTOR, 'input[data-cy="search.form.location.button"]'))
    # )

    # # Scroll into view and activate the field
    # dropdown_indicator = WebDriverWait(driver, 10).until(
    # EC.element_to_be_clickable((By.CSS_SELECTOR, '.react-select__indicators svg'))
    # )
    # dropdown_indicator.click()

    # driver.execute_script("arguments[0].scrollIntoView(true);", input_field)
    # input_field.click()

    # # Remove readonly attribute if it exists
    # if input_field.get_attribute("readonly"):
    #     driver.execute_script("arguments[0].removeAttribute('readonly')", input_field)

    # # Clear the field and input the desired string
    # input_field.clear()
    # input_field.send_keys(city)  # Replace with the actual string

    # # Verify the input was entered
    # print(f"Input field value: {input_field.get_attribute('value')}")
    # list_items = WebDriverWait(driver, 10).until(
    #     EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[role='listitem'].e12tw922"))
    # )

    # # Click the first element
    # if list_items:
    #     first_item = list_items[0]
    #     first_item.click()
    #     print("First item clicked successfully.")
    # else:
    #     print("No list items found.")
    # time.sleep(0.3)

    # # # Locate and click the confirm button
    # # button = WebDriverWait(driver, 10).until(
    # #     EC.element_to_be_clickable((By.ID, "confirm-location-search"))
    # # )
    # # # Click the button
    # # button.click()
    # # print("Button clicked successfully.")
    
    # button = WebDriverWait(driver, 10).until(
    #     EC.element_to_be_clickable((By.ID, "search-form-submit"))
    # )

    # button.click()
    # print("Button clicked successfully.")
    # time.slee(0.5)

    li_elements = driver.find_elements(By.TAG_NAME, 'li')
    hrefs = []
    # print(f"AAAAAAAAAAAAAAAAAAAAA{len(li_elements)}")
    for li in li_elements:
        try:
            # Find <a> tag with the specific data-cy attribute
            a_element = li.find_element(By.CSS_SELECTOR, 'a[data-cy="listing-item-link"]')
            href = a_element.get_attribute('href')  # Get the href attribute
            hrefs.append(href)  # Append href to the list
            # if len(hrefs)==nrOfElemns:
            #     break
        except Exception as e:
            continue  # Skip if no <a> element is found or any other error occurs
    # driver.quit()
    return hrefs
    

# def get_rent_data(rentLink):
#     driver=webdriver.Chrome()
#     rentData=pd.Series()
#     driver.get(rentLink)


#     rentData["pret"]=driver.find_element(By.CSS_SELECTOR, 'strong[data-cy="adPageHeaderPrice"]').text
#     rentData["adresa"]=driver.find_element(By.CSS_SELECTOR, 'a.css-1jjm9oe').text
#     rentData["suprafata"]=driver.find_elements(By.CLASS_NAME, 'css-1ftqasz')[0].text
#     rentData["numar_camere"]=driver.find_elements(By.CLASS_NAME, 'css-1ftqasz')[1].text
#     rentData["descriere"]= driver.find_element(By.CSS_SELECTOR, 'div[data-cy="adPageAdDescription"]').text
#     rentData["link"]=rentLink
#     rentData["autobuz"]=get_bus_distance(homeAddress=rentData["adresa"],officeAddress="Calea Floreasca 246c, București 077190")


#     # # Initialize variables
#     # nrColumns = 0
#     # p_elements = driver.find_elements(By.CLASS_NAME, 'e1qhas4i2.css-nlohq6')
#     # newText = ""
#     # # Iterate through the elements to gather information
#     # for elem in p_elements:
#     #     if ':' in elem.text:
#     #         nrColumns += 1
#     #     newText += elem.text + "\n"  # Adding a newline for better readability
#     #     # Populate the informatii Series
#     # for index in range(nrColumns):
#     #     indexHelp = index * 2
#     #     if indexHelp + 1 < len(p_elements):  # Check to avoid IndexError
#     #         key = p_elements[indexHelp].text[:-1]  # Remove the trailing colon
#     #         value = p_elements[indexHelp + 1].text  # Get the value
#     #         rentData[key] = value  # Set the key-value pair in the Series
    

#     # try:
#     #     consent_close_button = WebDriverWait(driver, 10).until(
#     #         EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))
#     #     )
#     #     consent_close_button.click()
#     # except Exception as e:
#     #     print("Consent popup not found or couldn't close:", e)

#     # # Now attempt to click the header button again
#     # headers = WebDriverWait(driver, 10).until(
#     #     EC.presence_of_all_elements_located((By.CLASS_NAME, 'css-1g1u77j'))
#     # )

#     # headers[0].click()
#     # headers[1].click()
#     # parent_element = driver.find_element(By.CLASS_NAME, 'css-gfsn8h')
#     # # Retrieve child elements with the specified class names
#     # child_elements = parent_element.find_elements(By.CSS_SELECTOR, '.e1qhas4i2.css-nlohq6')
#     # text=""
#     # for elem in child_elements:
#     #     text=text+elem.text

#     # nrColumns=0
#     # for elem in child_elements:
#     #     if ':' in elem.text:
#     #         nrColumns=nrColumns+1
#     # for index in range(nrColumns):
#     #     indexHelp = index * 2
#     #     if indexHelp + 1 < len(child_elements):  # Check to avoid IndexError
#     #         key = child_elements[indexHelp].text[:-1]  # Remove the trailing colon
#     #         value = child_elements[indexHelp + 1].text  # Get the value
#     #         rentData[key] = value  # Set the key-value pair in the Series
#     # container = WebDriverWait(driver, 10).until(
#     # EC.presence_of_element_located((By.CLASS_NAME, 'css-drw7ek'))
#     # )
#     # texts = container.text
#     # print(texts)
#     # nrColumns=texts.count(':')
#     # newText=""
#     # for line in texts.splitlines():
#     #     if(':' in line):
#     #         rentData[line[:-1]]=None
#     #         aux=line[:-1]
#     #         newText=""
#     #     else:
#     #         newText=newText+","+line
#     #         rentData[aux]=newText


    
#     # rentData[driver.find_elements(By.CLASS_NAME,"css-t7cajz e1qhas4i1")[0]]=driver.find_elements(By.CLASS_NAME,"css-511gym")[0]


    
#     driver.quit()
#     return rentData

#     # elements=driver.find_elements(By.CLASS_NAME, 'e1qhas4i2.css-nlohq6')
#     # extra_info = [element.text for element in elements[12:]]
#     # rentData["extra_info"]=extra_info


#     # rentData["an_constructie"]=None
#     # rentData["lift"]=None
#     # rentData["tip_cladire"]=None
#     # rentData["material_constructie"]=None
#     # rentData["tip_geamuri"]=None
#     # rentData["facilitati"]=None
#     # rentData["siguranta"]=None
#     # rentData["media"]=None
def get_attributes(driver):
    extra_attributes = {}
    p_elements = driver.find_elements(By.XPATH, "//p[contains(@class, 'css-nlohq6')]")
    p_elements_text=[p_element.text for p_element in p_elements]
    # for index in range(0, len(p_elements), 2):
    #     try:
    #         if p_elements[index].text[:-1]!="" and p_elements[index+1].text[:-1]!="":
    #             key = p_elements[index].text[:-1]  # Remove the trailing colon from the key
    #             value = p_elements[index + 1].text  # The next element is the value
    #             extra_attributes[key] = value  # Store the attribute-value pair
    #         print(key,value)
    #     except IndexError:
    #             continue  # Skip if there's an odd number of elements
    return p_elements_text
def click_with_fallback(element, description,driver):
    try:
        # Scroll into view
        driver.execute_script("arguments[0].scrollIntoView(true);", element)

        # Wait until clickable and click
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable(element))
        element.click()
        print(f"{description} clicked successfully!")
    except (ElementClickInterceptedException, TimeoutException):
        # Fallback to JavaScript click
        print(f"{description} could not be clicked directly. Attempting JavaScript click...")
        try:
            driver.execute_script("arguments[0].click();", element)
            print(f"{description} clicked via JavaScript!")
        except Exception as js_error:
            print(f"JavaScript click failed for {description}: {js_error}")

def get_rent_data(rentLink,address):
    
    driver = webdriver.Chrome()
    rentData = pd.Series()  # Create an empty Series to store the data
    rentData["materiale_cladire"]=None
    rentData["facilitati_apartament"]=None
    driver.get(rentLink)
    extraAtrributes1=get_attributes(driver)
    rentData["extra_apartament"]=extraAtrributes1
    try:
        accept_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))
        )
        click_with_fallback(accept_button, "Accept cookies button",driver)
    except TimeoutException:
        print("No 'Accept' button found or already dismissed.")
    try:
        # Find all buttons by class name
        header_buttons = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "css-1g1u77j"))
        )

        print(f"Found {len(header_buttons)} buttons. Clicking each...")
        time.sleep(0.5)
        # Loop through and click each button
        for index, button in enumerate(header_buttons):
            try:
                # Scroll into view (useful if the button is not visible)
                driver.execute_script("arguments[0].scrollIntoView(true);", button)

                # Ensure the button is clickable
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable(button))

                # Click the button
                button.click()
                
                if index==0:
                    extraAtrributes2=get_attributes(driver)
                    list1 = [value for value in extraAtrributes2 if value not in extraAtrributes1]
                    rentData["materiale_cladire"]=list1
                elif index==1:
                    extraAtrributes3=get_attributes(driver)
                    list2 = [value for value in extraAtrributes3 if value not in extraAtrributes2]
                    rentData["facilitati_apartament"]=list2
                    
                print(f"Button {index + 1} clicked successfully!")


            except (TimeoutException, ElementClickInterceptedException):
                print(f"Button {index + 1} could not be clicked.")

    except Exception as e:
        print(f"An error occurred: {e}")

    try:
        # Extracting standard attributes
        rentData["pret"] = driver.find_element(By.CSS_SELECTOR, 'strong[data-cy="adPageHeaderPrice"]').text
        rentData["adresa"] = driver.find_element(By.CSS_SELECTOR, 'a.css-1jjm9oe').text
        rentData["suprafata"] = driver.find_elements(By.CLASS_NAME, 'css-1ftqasz')[0].text
        rentData["numar_camere"] = driver.find_elements(By.CLASS_NAME, 'css-1ftqasz')[1].text
        rentData["descriere"] = driver.find_element(By.CSS_SELECTOR, 'div[data-cy="adPageAdDescription"]').text
        rentData["link"] = rentLink
        timp=get_bus_distance(homeAddress=rentData["adresa"], officeAddress=address)
        rentData["autobuz"] = timp
        print(timp)
        # Extracting additional dynamic attributes


    except Exception as e:
        print(f"Error extracting rent data: {e}")
    
    driver.quit()
    return rentData

    

    





In [62]:
if False:
    from opencage.geocoder import OpenCageGeocode
    from geopy.distance import geodesic
    load_dotenv()
    # Function to get coordinates of an address
    def get_coords(address, api_key):
        geocoder = OpenCageGeocode(api_key)
        result = geocoder.geocode(address)
        if result and len(result):
            return (result[0]['geometry']['lat'], result[0]['geometry']['lng'])
        return None

    # Function to calculate distance between two addresses
    def calculate_distance(address1, address2, api_key):
        coords_1 = get_coords(address1, api_key)
        coords_2 = get_coords(address2, api_key)
        
        if coords_1 and coords_2:
            # Calculate distance in kilometers using geodesic
            distance_km = geodesic(coords_1, coords_2).kilometers
            return distance_km
        return None

    # Example usage

    address1 = "1600 Amphitheatre Parkway, Mountain View, CA"
    address2 = "1 Infinite Loop, Cupertino, CA"

    distance = calculate_distance(address1, address2, os.getenv("api_key"))
    if distance:
        print(f"Distance: {distance} km")
    else:
        print("Failed to retrieve data")


In [63]:
if False:    # First, load the CSV data
    rentData = pd.read_csv("chirii.csv")

    # Get coordinates of the office address
    coordonateBirou = get_coords(address="calea floreasca 246c bucuresti", api_key=os.getenv("api_key"))

    # Loop through the dataset and calculate distances
    for index, record in rentData.iterrows():
        print(index)
        # Get coordinates of the current address
        # coords_address = get_coords(record["adresa"], api_key)
        coords_address=record["adresa"]
        # If coordinates are available, calculate the distance
        if coords_address:
            distance = calculate_distance("calea floreasca 246c bucuresti", coords_address, os.getenv("api_key"))
            rentData.at[index, "distance"] = distance
        else:
            rentData.at[index, "distance"] = None  # Handle cases where geocoding fails

    # Save the updated data with distances to a new CSV file
    rentData.to_csv("chirii_with_distances.csv", index=False)


In [64]:
if False:
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.common.exceptions import NoSuchElementException

    def get_div_text(driver, url):
        driver.get(url)
        div_element = driver.find_element(By.CSS_SELECTOR, 'div[jstcache="522"]')
        div_text = div_element.text
        return div_text

    # Example usage:
    driver = webdriver.Chrome()  # or the browser of your choice
    url = "https://www.google.com/maps/dir/Bulevardul+Tineretului,+Bucure%C8%99ti/Calea+Floreasca+246c,+Bucure%C8%99ti+077190/@44.4439973,26.0671406,13z/data=!3m1!4b1!4m14!4m13!1m5!1m1!1s0x40b1fefe144ede77:0x5c99dd4ae925a749!2m2!1d26.1107837!2d44.410399!1m5!1m1!1s0x40b20388e10cc5ab:0x252c88de294519b6!2m2!1d26.1054352!2d44.4774927!3e2?entry=ttu&g_ep=EgoyMDI0MTAxNi4wIKXMDSoASAFQAw%3D%3D"
    try:
        accept_button = driver.find_element(By.XPATH, "//button[.//span[text()='Accept all']]")
        accept_button.click()
        print("Button clicked successfully!")
    except NoSuchElementException:
        # If the button doesn't exist
        print("Button not found.")
    print(get_div_text(driver, url))

    # Don't forget to close the browser after you're done
    driver.quit()


In [65]:
if False:
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC

    # Assuming the webdriver is already set up
    driver = webdriver.Chrome()

    # Open your target page
    driver.get("https://www.google.com/maps/dir/Bulevardul+Tineretului,+Bucure%C8%99ti/Calea+Floreasca+246c,+Bucure%C8%99ti+077190/@44.4439973,26.0671406,13z/data=!3m1!4b1!4m14!4m13!1m5!1m1!1s0x40b1fefe144ede77:0x5c99dd4ae925a749!2m2!1d26.1107837!2d44.410399!1m5!1m1!1s0x40b20388e10cc5ab:0x252c88de294519b6!2m2!1d26.1054352!2d44.4774927!3e2?entry=ttu&g_ep=EgoyMDI0MTAxNi4wIKXMDSoASAFQAw%3D%3D")

    try:
        # Wait for the button to be visible and clickable
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[.//span[text()='Accept all']]"))
        )    
        try:
            accept_button.click()
            print("Button clicked successfully!")
        except (ElementClickInterceptedException, ElementNotInteractableException):
            # Fallback to JavaScript click in case of issues
            driver.execute_script("arguments[0].click();", accept_button)
            print("Button clicked using JavaScript!")
    except NoSuchElementException:
        # If the button doesn't exist
        print("Button not found.")

    try:
        travel_mode_div = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '//div[@data-travel_mode="3"]'))
        )
        travel_mode_text = travel_mode_div.text
        
        print("Text inside the div with data-travel_mode=3:")
        print(travel_mode_text)

    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.quit()


In [66]:
if False:
    import pandas as pd
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException, TimeoutException
    from selenium.webdriver.common.keys import Keys

    # Load the rent data
    rentData = pd.read_csv("chirii.csv")
    adresaBirou = "Calea Floreasca 246c, București 077190"

    # Initialize the WebDriver
    driver = webdriver.Chrome()  # or webdriver.Firefox(), etc.

    # Open Google Maps
    driver.get("https://www.google.ro/maps/preview/")

    # Function to click an element with fallback to JavaScript
    def click_element(locator):
        try:
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(locator)
            )
            element.click()
            print("Button clicked successfully!")
        except (ElementClickInterceptedException, ElementNotInteractableException):
            driver.execute_script("arguments[0].click();", element)
            print("Button clicked using JavaScript!")
        except NoSuchElementException:
            print("Element not found.")

    # Accept the cookies button
    try:
        accept_button_locator = (By.XPATH, "//button[.//span[text()='Accept all']]")
        click_element(accept_button_locator)
    except NoSuchElementException:
        print("Accept button not found.")

    # Input the address in the search box
    try:
        search_input = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, "searchboxinput"))
        )
        search_input.clear()
        search_input.send_keys(adresaBirou)
        print(f"Inserted '{adresaBirou}' into the search box.")

        # Click the search button
        search_button_locator = (By.ID, "hArJGc")
        click_element(search_button_locator)
        print("The search button is clicked with 1 argument.")

        # Input a different address
        input_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "tactile-searchbox-input"))
        )
        input_field.clear()
        homeAddress = "Bulevardul Tineretului, Tineretului, Sectorul 4, Bucuresti"
        input_field.send_keys(homeAddress)
        print(f"Inserted '{homeAddress}' into the search box.")
        input_field.send_keys(Keys.RETURN)
        print("The final search button has been clicked.")

        # Wait for the travel mode div to be visible and extract text
        travel_mode_div = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '//div[@data-travel_mode="3"]'))
        )
        travel_mode_text = travel_mode_div.text
        print(travel_mode_div.get_attribute("outerHTML"))
        print("Text inside the div with data-travel_mode=3:")
        print(travel_mode_text)

    except TimeoutException as te:
        print(f"Timeout Exception: {te}")
    except Exception as e:
        print(f"Error: {e}")

    # Close the driver when done
    driver.quit()


In [67]:
def get_user_input():
    while True:
        try:
            # Get an integer from the user
            num_rents = int(input("Enter the number of rents to search (an integer): "))
            
            # Get a string for the address
            address = input("Enter the address (e.g., '123 Main St'): ").strip()
            if not address:
                raise ValueError("Address cannot be empty.")
            
            # Get a string for the city
            city = input("Enter the city: ").strip()
            if not city:
                raise ValueError("City cannot be empty.")
            
            # Return the inputs as a tuple
            return num_rents, address, city
        except ValueError as e:
            print(f"Invalid input: {e}. Please try again.")

# Example usage



In [68]:
import pandas as pd

# Create an empty list to store data
all_rent_data = []

links=-1
while links==-1:
    values=get_user_input()
    links = get_rent_links(1,values[0],values[2])  # Get rent links for page 5 (or any other page)
for link in links:
    data = get_rent_data(link,values[1])  # Get the data for each link
    all_rent_data.append(data)  # Append the data to the list

# Convert the list of rent data into a DataFrame
df = pd.DataFrame(all_rent_data)

# Append the DataFrame to a CSV file
df.to_csv('rent_data.csv',index=False)



brasov
brasov
Accept cookies button clicked successfully!


In [69]:
import ftfy
import csv
rentData=pd.read_csv("rent_data.csv")
def format_autobuz_time(time_str):
    # Initialize total minutes
    total_minutes = 0
    
    # Check for hours
    if "h" in time_str or "hr" in time_str:
        # Get hours; handle both "h" and "hr"
        if "h" in time_str:
            hours = int(time_str.split("h")[0].strip())  # Get hours
        else:
            hours = int(time_str.split("hr")[0].strip())  # Get hours
        total_minutes += hours * 60  # Convert hours to minutes
    
    # Check for minutes
    if "m" in time_str or "min" in time_str:
        # Get minutes; handle both "m" and "min"
        if "m" in time_str:
            minutes = int(time_str.split("m")[0].split()[-1].strip())  # Get minutes
        else:
            minutes = int(time_str.split("min")[0].split()[-1].strip())  # Get minutes
        total_minutes += minutes  # Add minutes
    
    if(total_minutes!=0):
        return total_minutes
    else:
        return time_str
    
    

try:
    # Assume rentData is a DataFrame
    rentData = rentData.map(lambda x: ftfy.fix_text(str(x)).strip())  # Clean text

    # Clean 'descriere' column
    rentData['descriere'] = rentData['descriere'].apply(
        lambda x: x.replace('\n', ' ').replace('\r', ' ').strip()
    )

    # Clean 'pret' column (price): Remove euro symbol (€), spaces
    rentData["pret"] = rentData['pret'].apply(
        lambda x: x.replace("€", '').replace(" ", '').strip()
    )

    # Clean 'suprafata' column: Remove 'm²'
    rentData["suprafata"] = rentData["suprafata"].apply(
        lambda x: x.replace("m²", '').strip()
    )

    # Format 'autobuz' column (example placeholder function)
    def format_autobuz_time(x):
        try:
            return x.strip() + " min"  # Example formatting, adapt as needed
        except AttributeError:
            return x  # Return original value if issues occur

    rentData['autobuz'] = rentData['autobuz'].apply(format_autobuz_time)

    # Save to CSV
    rentData.to_csv(
        "chirii2.csv",
        index=False,
        encoding="utf-8-sig",
        sep=',',
        quotechar='"',
        quoting=csv.QUOTE_ALL
    )

    print("Data successfully cleaned and saved to chirii2.csv!")

except ValueError as e:
    print(f"ValueError occurred: {e}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")



# for index,record in rentData.iterrows():
#     rentData.at[index,"score"]=-3*record["pret"]+2*record["suprafata"]-2*record["autobuz"]


EmptyDataError: No columns to parse from file

In [11]:
# sorted_df = rentData.sort_values(by=["autobuz", "pret"], ascending=[True, True])

# # Save the sorted data to a new CSV if needed
# sorted_df.to_csv("sorted_chirii.csv", index=False)